In [1]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd

project = 'possible-coast-463614-i9' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Extract the current year
from datetime import datetime
current_year = datetime.now().year
current_year

2025

In [4]:
# Function to execute a BigQuery query and return a DataFrame

def query_to_dataframe(query: str) -> pd.DataFrame:
    """
    Executes a SQL query in BigQuery and returns a Pandas DataFrame.

    Parameters:
    - query (str): The SQL query to execute.

    Return:
    - pd.DataFrame : The DataFrame containing the results of the query.
    """
    try:
        df = client.query(query).to_dataframe()
        print(f"Query executed successfully. Retrieved {df.shape[0]} rows.")
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()

# III/ Financial & Pricing Analysis



## Question 7: How does the total fare revenue of yellow taxis change over time?

In [5]:
query_total_fare_revenue_over_time = """
SELECT *
FROM `possible-coast-463614-i9.views_fordashboard.total_fare_revenue_over_time`
"""
total_fare_revenue_over_time_df = query_to_dataframe(query_total_fare_revenue_over_time)
total_fare_revenue_over_time_df.head()

Query executed successfully. Retrieved 1266 rows.


,trip_date,year,month,week,weekday,total_revenue,fare_revenue,tip_revenue,tolls_revenue,congestion_revenue
0,2023-06-01,2023,6,22,5,3410868.26,2327436.58,424764.27,64504.16,268810.0
1,2023-06-21,2023,6,25,4,3231328.35,2197181.60,406152.45,61484.80,254875.0
2,2023-03-19,2023,3,12,1,2799254.51,1946780.81,351532.53,72912.22,215912.5
3,2024-03-30,2024,3,12,7,2422812.73,1670778.45,298821.78,43205.96,216247.5
4,2024-03-01,2024,3,8,6,2887660.47,1924930.67,365856.10,57550.00,242835.0


In [6]:
# Filter rows where the year is between 2020 and the current year (inclusive)
filtered_total_fare_revenue_over_time_df = total_fare_revenue_over_time_df[(total_fare_revenue_over_time_df['year'] >= 2020) & (total_fare_revenue_over_time_df['year'] <= current_year)]
filtered_total_fare_revenue_over_time_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1249 entries, 0 to 1265
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   trip_date           1249 non-null   dbdate 
 1   year                1249 non-null   Int64  
 2   month               1249 non-null   Int64  
 3   week                1249 non-null   Int64  
 4   weekday             1249 non-null   Int64  
 5   total_revenue       1249 non-null   float64
 6   fare_revenue        1249 non-null   float64
 7   tip_revenue         1249 non-null   float64
 8   tolls_revenue       1249 non-null   float64
 9   congestion_revenue  1249 non-null   float64
dtypes: Int64(4), dbdate(1), float64(5)
memory usage: 112.2 KB


In [7]:
# Create an area chart
fig = px.area(
    filtered_total_fare_revenue_over_time_df,
    x="trip_date",
    y=["fare_revenue", "tip_revenue", "tolls_revenue", "congestion_revenue"],
    labels={"trip_date": "Date", "value": "Revenue ($)"},
    title="NYC Taxi Revenue Trends Over Time",
    color_discrete_sequence=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"],
)

fig.show()


In [8]:
# Aggregate revenue by month
monthly_revenue = filtered_total_fare_revenue_over_time_df.groupby(["year", "month"])[
    ["fare_revenue", "tip_revenue", "tolls_revenue", "congestion_revenue"]
].sum().reset_index()

# Convert year-month to a single string for better x-axis readability
monthly_revenue["year_month"] = monthly_revenue["year"].astype(str) + "-" + monthly_revenue["month"].astype(str).str.zfill(2)

# Create the stacked bar chart
fig = px.bar(
    monthly_revenue,
    x="year_month",
    y=["fare_revenue", "tip_revenue", "tolls_revenue", "congestion_revenue"],
    title="Monthly Revenue Breakdown",
    labels={"value": "Revenue ($)", "year_month": "Year-Month"},
    color_discrete_sequence=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"],
)

fig.show()

In [ ]:
import plotly.io as pio

# Set the renderer to "colab" for Google Colab
pio.renderers.default = "colab"

# Assuming 'filtered_total_fare_revenue_over_time_df' is already loaded
df = filtered_total_fare_revenue_over_time_df.copy()

# Add a weekend column if not present
if "is_weekend" not in df.columns:
    df["is_weekend"] = df["weekday"].isin([5, 6])  # Assuming 5=Saturday, 6=Sunday

# Define granularity options and weekend filters
time_granularity_options = ["day", "week", "month", "year"]
weekend_filter_options = [("All", None), ("Weekends", True), ("Weekdays", False)]

# Function to generate and display a chart
def generate_plot(granularity, weekend_status):
    agg_column = "trip_date"  # Default to daily granularity
    if granularity == "week":
        agg_column = "week"
    elif granularity == "month":
        agg_column = "month"
    elif granularity == "year":
        agg_column = "year"

    # Filter by weekend status
    filtered_df = df.copy()
    if weekend_status is not None:
        filtered_df = filtered_df[filtered_df["is_weekend"] == weekend_status]

    # Aggregate total revenue
    revenue_df = filtered_df.groupby(agg_column)["total_revenue"].sum().reset_index()

    # Plot
    fig = px.line(
        revenue_df,
        x=agg_column,
        y="total_revenue",
        title=f"📈 Total Revenue Over Time ({granularity}, {'All' if weekend_status is None else 'Weekends' if weekend_status else 'Weekdays'})",
        labels={"total_revenue": "Total Revenue ($)", agg_column: granularity.capitalize()},
        line_shape="spline",
        markers=True
    )
    fig.show()

# Loop through all combinations and display the charts
for granularity in time_granularity_options:
    for label, weekend_status in weekend_filter_options:
        generate_plot(granularity, weekend_status)


## Question 8: What is the average fare per trip, and how does it vary by borough, time of day, and trip distance?

In [ ]:
query_avg_fare_analysis = """
SELECT *
FROM `nyc-yellow-trips.views_fordashboard.avg_fare_analysis`
"""
avg_fare_analysis_df = query_to_dataframe(query_avg_fare_analysis)
avg_fare_analysis_df.head()

Query executed successfully. Retrieved 906483 rows.


,trip_date,year,month,pickup_hour,pickup_borough,dropoff_borough,avg_fare_per_trip,avg_total_amount_per_trip,avg_trip_distance,total_trips
0,2020-08-31,2020,8,16,Manhattan,Brooklyn,30.39,39.53,6.81,50
1,2020-08-22,2020,8,16,Manhattan,Manhattan,9.22,14.09,2.07,1785
2,2020-08-06,2020,8,11,Manhattan,Manhattan,9.62,14.46,1.89,1892
3,2020-08-27,2020,8,20,Manhattan,Manhattan,8.68,14.23,1.92,1345
4,2020-08-24,2020,8,7,Manhattan,Manhattan,8.78,13.58,1.98,939


In [ ]:
# Filter rows where the year is between 2020 and the current year (inclusive)
filtered_avg_fare_analysis_df = avg_fare_analysis_df[(avg_fare_analysis_df['year'] >= 2020) & (avg_fare_analysis_df['year'] <= current_year)]
filtered_avg_fare_analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 906121 entries, 0 to 906482
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   trip_date                  906121 non-null  dbdate 
 1   year                       906121 non-null  Int64  
 2   month                      906121 non-null  Int64  
 3   pickup_hour                906121 non-null  Int64  
 4   pickup_borough             906121 non-null  object 
 5   dropoff_borough            906121 non-null  object 
 6   avg_fare_per_trip          906121 non-null  float64
 7   avg_total_amount_per_trip  906121 non-null  float64
 8   avg_trip_distance          906121 non-null  float64
 9   total_trips                906121 non-null  Int64  
dtypes: Int64(4), dbdate(1), float64(3), object(2)
memory usage: 79.5+ MB


In [ ]:
# 1. Heatmap: Analyze average fare by pickup and dropoff borough
fig_heatmap = px.density_heatmap(
    filtered_avg_fare_analysis_df,
    x="pickup_borough",
    y="dropoff_borough",
    z="avg_fare_per_trip",
    color_continuous_scale="Viridis",
    title="Heatmap of Average Fare per Trip by Pickup and Dropoff Borough",
    labels={"pickup_borough": "Pickup Borough", "dropoff_borough": "Dropoff Borough", "avg_fare_per_trip": "Average Fare ($)"},
)
fig_heatmap.show()

In [ ]:
# 2. Bar Chart: Average fare per trip by hour of the day
hourly_avg_fare = filtered_avg_fare_analysis_df.groupby("pickup_hour")["avg_fare_per_trip"].mean().reset_index()
fig_bar = px.bar(
    hourly_avg_fare,
    x="pickup_hour",
    y="avg_fare_per_trip",
    title="Average Fare per Trip by Hour of the Day",
    labels={"pickup_hour": "Pickup Hour", "avg_fare_per_trip": "Average Fare ($)"},
    text_auto=True,
)
fig_bar.update_traces(marker_color="blue", textfont_size=12)
fig_bar.show()

In [ ]:
# 3. Scatter Plot: Average fare vs. average trip distance
fig_scatter = px.scatter(
    filtered_avg_fare_analysis_df,
    x="avg_trip_distance",
    y="avg_fare_per_trip",
    size="total_trips",
    color="pickup_borough",
    title="Average Fare vs. Average Trip Distance (by Pickup Borough)",
    labels={"avg_trip_distance": "Average Trip Distance (miles)", "avg_fare_per_trip": "Average Fare ($)", "total_trips": "Total Trips"},
    hover_data=["pickup_borough", "dropoff_borough"],
)
fig_scatter.show()

## Question 9: What is the proportion of different payment types (credit card, cash, etc.), and has it changed over time?

In [ ]:
query_payment_type_trends = """
SELECT *
FROM `nyc-yellow-trips.views_fordashboard.payment_type_trends`
"""
payment_type_trends_df = query_to_dataframe(query_payment_type_trends)
payment_type_trends_df.head()

Query executed successfully. Retrieved 7225 rows.


,trip_date,year,month,week,payment_method,total_trips,daily_total_trips,payment_percentage
0,2020-02-29,2020,2,8,Credit Card,164046,222222,73.82
1,2020-02-29,2020,2,8,No Charge,765,222222,0.34
2,2020-02-29,2020,2,8,Dispute,301,222222,0.14
3,2020-02-29,2020,2,8,Cash,57110,222222,25.70
4,2020-03-03,2020,3,9,Cash,48064,213760,22.49


In [ ]:
# Filter rows where the year is between 2020 and the current year (inclusive)
filtered_payment_type_trends_df = payment_type_trends_df[(payment_type_trends_df['year'] >= 2020) & (payment_type_trends_df['year'] <= current_year)]
filtered_payment_type_trends_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7185 entries, 0 to 7224
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   trip_date           7185 non-null   dbdate 
 1   year                7185 non-null   Int64  
 2   month               7185 non-null   Int64  
 3   week                7185 non-null   Int64  
 4   payment_method      7185 non-null   object 
 5   total_trips         7185 non-null   Int64  
 6   daily_total_trips   7185 non-null   Int64  
 7   payment_percentage  7185 non-null   float64
dtypes: Int64(5), dbdate(1), float64(1), object(1)
memory usage: 540.3+ KB


In [ ]:
# Ensure 'trip_date' is in datetime format if not already
filtered_payment_type_trends_df["trip_date"] = pd.to_datetime(filtered_payment_type_trends_df["trip_date"])

<ipython-input-21-9b61fb5051a3>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Aggregate data to calculate the overall proportion of payment methods
payment_summary = filtered_payment_type_trends_df.groupby("payment_method")["total_trips"].sum().reset_index()
payment_summary["percentage"] = (
    payment_summary["total_trips"] / payment_summary["total_trips"].sum()
) * 100

# Create a donut chart
fig_donut = px.pie(
    payment_summary,
    names="payment_method",
    values="percentage",
    title="Proportion of Payment Methods",
    hole=0.5,  # This creates the donut chart effect
    labels={"payment_method": "Payment Method", "percentage": "Percentage (%)"},
)

fig_donut.update_traces(
    textinfo="percent+label",  # Show percentage and label
    hoverinfo="label+percent+value",  # Tooltip details
    pull=[0.1 if p == payment_summary["percentage"].max() else 0 for p in payment_summary["percentage"]],  # Emphasize the largest segment
)

fig_donut.update_layout(
    showlegend=True,
    legend_title_text="Payment Methods",
    template="plotly_white",
)

fig_donut.show()


In [ ]:
fig = px.area(
    filtered_payment_type_trends_df,
    x="trip_date",
    y="payment_percentage",
    color="payment_method",
    title="Proportion of Different Payment Methods Over Time",
    labels={"payment_percentage": "Payment Share (%)", "trip_date": "Date"},
    template="plotly_white"
)

fig.show()